<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/10_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos

Vamos a usar el conocido data set 20 Newsgroup. Se trata de un conjunto de datos clásico y muy utilizado en el campo del procesamiento de lenguaje natural (NLP) y el aprendizaje automático. Fue recopilado originalmente por Ken Lang en 1995 y contiene aproximadamente 20.000 mensajes tomados de 20 grupos de noticias (newsgroups) diferentes de Usenet, una red de foros de discusión en línea muy popular antes del auge de la web moderna. Viene cargado en Sklearn.

In [1]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

Vamos a tener que instalar bertopic porque no viene preinstalado en Colab

In [3]:
%%capture
!pip install bertopic

Luego, directamente entrenamos el modelo a ver que nos da.

# Bertopic

In [4]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

2025-05-15 17:26:48,199 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2025-05-15 17:53:37,953 - BERTopic - Embedding - Completed ✓
2025-05-15 17:53:37,955 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-15 17:54:22,827 - BERTopic - Dimensionality - Completed ✓
2025-05-15 17:54:22,833 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-15 17:55:19,400 - BERTopic - Cluster - Completed ✓
2025-05-15 17:55:19,422 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-15 17:55:23,103 - BERTopic - Representation - Completed ✓


La salida del modelo no asigna una etiqueta a cada tópico pero nos muestra las palabras más relevantes. En base a esa información uno podría asignarle nombres a los tópicos que le interese. El tópico -1 es el de los outliers, sería un tópico no relevante.

In [5]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6955,-1_to_the_of_is,"[to, the, of, is, you, and, for, it, in, this]","[A listmember (D Andrew Killie, I think) wrote..."
1,0,1827,0_game_team_games_he,"[game, team, games, he, players, season, hocke...",[1992-93 Los Angeles Kings notes and game repo...
2,1,562,1_key_clipper_chip_encryption,"[key, clipper, chip, encryption, keys, escrow,...","[April 16, 1993\n\nINITIAL EFF ANALYSIS OF CLI..."
3,2,529,2_ites_hello_cheek_hi,"[ites, hello, cheek, hi, yep, huh, ken, why, e...","[Hi,, \n \n ..."
4,3,486,3_israel_israeli_jews_arab,"[israel, israeli, jews, arab, jewish, arabs, p...","[\nThis a ""tried and true"" method utilized by ..."


Atributos
Hay varios atributos a los que podés acceder después de haber entrenado tu modelo BERTopic:

|Atributo	 | Descripción |
|------------------------|---------------------------------------------------------------------------------------------|
|topics_	|Los temas que se generan para cada documento después de entrenar o actualizar el modelo de temas.
|probabilities_	|Las probabilidades que se generan para cada documento si se utiliza HDBSCAN.
|topic_sizes_	|El tamaño de cada tema.
|topic_mapper_ |	Una clase para rastrear los temas y sus asignaciones cada vez que se combinan o reducen.
|topic_representations_	|Los n términos principales por tema y sus respectivos valores de c-TF-IDF.
|c_tf_idf_|	La matriz de términos por tema calculada mediante c-TF-IDF.
|topic_labels_|	Las etiquetas predeterminadas para cada tema.
|custom_labels_	|Etiquetas personalizadas para cada tema, generadas mediante .set_topic_labels.
|topic_embeddings_	|Los embeddings de cada tema si se utilizó embedding_model.
|representative_docs_	|Los documentos representativos de cada tema si se utilizó HDBSCAN.

Por ejemplo, para acceder a los temas predichos de los primeros 10 documentos, simplemente ejecutamos lo siguiente:

In [6]:
topic_model.topics_[:10]

[0, 165, 48, 40, 107, 189, -1, 0, 0, -1]

# Visualización

In [7]:
topic_model.visualize_topics()

La variable probabilities, que se devuelve al ejecutar transform() o fit_transform(), se puede usar para entender con qué grado de confianza BERTopic asigna uno o más temas a un documento.

Es decir, no solo te dice qué tema es el más probable, sino qué tan probable es cada tema dentro de ese documento. Esta información puede ser útil si un documento está relacionado con varios temas o si hay baja certeza sobre la clasificación.

Para visualizar cómo se distribuyen esas probabilidades (es decir, qué temas aparecen y con qué peso en un documento específico), simplemente ejecutás:

In [8]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

probs[200] indica que estás visualizando la distribución de temas del documento número 200.

min_probability=0.015 filtra los temas que tienen menos de 1.5% de probabilidad para que el gráfico sea más claro (oculta temas irrelevantes o con peso muy bajo).

Esto genera un gráfico interactivo (usualmente un gráfico de barras) que muestra qué temas aparecen en ese documento y con qué fuerza.

# Jerarquía de Tópicos

Los temas que fueron generados por BERTopic pueden reducirse jerárquicamente, es decir, pueden agruparse en temas más generales.

Para entender mejor la posible estructura jerárquica de los temas (por ejemplo, cómo unos temas están relacionados o se pueden combinar), se puede usar la función scipy.cluster.hierarchy para:

Agrupar los temas en clústeres jerárquicos.

Visualizar cómo se conectan o agrupan esos temas entre sí.

Esto puede ayudarte a decidir cuántos temas finales (nr_topics) querés mantener cuando reduzcas el número total de temas generados automáticamente por el modelo.

Por ejemplo, si BERTopic detectó inicialmente 50 temas pero muchos son muy parecidos, esta visualización puede ayudarte a ver que tal vez con 20 o 25 temas sería suficiente.

In [9]:
topic_model.visualize_hierarchy(top_n_topics=50)

# Visualización de términos

Podemos visualizar los términos seleccionados de algunos temas creando gráficos de barras a partir de los puntajes de c-TF-IDF de cada representación de tema.

El c-TF-IDF (class-based TF-IDF) es una versión adaptada del TF-IDF tradicional, que mide la importancia de cada palabra dentro de un tema (en lugar de dentro de un documento).

Así, los términos con mayor c-TF-IDF son los más representativos de ese tema.

Esto permite:

Obtener insights (ideas clave) observando los puntajes relativos de c-TF-IDF dentro de un tema (qué términos lo definen más claramente) y entre temas (qué los diferencia).

Comparar de forma sencilla las representaciones de distintos temas entre sí, viendo qué palabras los definen y en qué grado.

In [10]:
topic_model.visualize_barchart(top_n_topics=5)

# Similitud de tópicos

Una vez que se han generado los embeddings de los temas (representaciones numéricas de cada tema), ya sea mediante c-TF-IDF o mediante un modelo de embeddings, se puede construir una matriz de similitud aplicando la similitud del coseno entre esos vectores.

Esto produce una matriz que indica qué tan similares son los distintos temas entre sí.

Cada celda de la matriz muestra el grado de similitud (entre -1 y 1, aunque en la práctica será entre 0 y 1) entre dos temas.

Valores más cercanos a 1 indican que esos temas son muy parecidos en su contenido.

Esta matriz es útil, por ejemplo, para identificar temas redundantes o agrupar temas relacionados.

In [11]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

# Personalización

Uno de los puntos más fuertes de BERTopic es su modularidad: podés combinar diferentes técnicas de vectorización, embeddings, reducción de dimensionalidad, clustering, e incluso asignación de tópicos. Al cambiar estos módulos (y no solo sus hiperparámetros), podés obtener resultados más adecuados a la naturaleza de tus textos y al tipo de análisis que querés hacer.

🧩 ¿Qué componentes del modelo BERTopic se pueden personalizar?

📌 1. Vectorización de texto
Convierte los documentos en matrices de términos. Algunas alternativas:

CountVectorizer (por defecto)

TfidfVectorizer (mejor si hay mucho ruido o vocabulario disperso)

Vectorizadores con n-gramas, eliminación de stopwords personalizados o reducción de vocabulario (max_df, min_df)

Incluso se puede usar un vectorizador preprocesado externo si tenés un pipeline específico

📌 2. Modelo de embeddings
Convierte el texto en vectores semánticos. Algunas opciones:

all-MiniLM-L6-v2 (rápido y balanceado, por defecto)

all-mpnet-base-v2 (más preciso, más pesado)

distiluse-base-multilingual-cased (para textos en múltiples idiomas)

Modelos propios entrenados con sentence-transformers

Alternativas no basadas en BERT, como GloVe o spaCy (menos comunes)

📌 3. Reducción de dimensionalidad
Reduce la dimensionalidad de los embeddings para facilitar el clustering:

UMAP (por defecto, no lineal, configurable con n_neighbors, min_dist, etc.)

PCA (lineal, útil cuando los embeddings ya son de baja dimensión)

TSNE (más costoso pero visualmente informativo)

También podés omitir esta etapa si no es necesaria

📌 4. Modelo de clustering
Agrupa documentos similares según sus vectores reducidos:

HDBSCAN (por defecto, no requiere predefinir el número de clusters, tolerante al ruido)

KMeans (requiere n_clusters, útil para comparación directa)

AgglomerativeClustering (jerárquico, útil si esperás relaciones entre grupos)

SpectralClustering, DBSCAN o modelos propios de clustering también se pueden usar

📌 5. Asignación y representación de tópicos
BERTopic también permite:

Cambiar el número de tópicos finales (nr_topics)

Fusionar tópicos jerárquicamente

Cambiar la representación textual de los tópicos (set_topic_labels, set_topic_representations)

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

# 1. Vectorizador alternativo: TF-IDF en lugar de CountVectorizer
vectorizer_model = TfidfVectorizer(ngram_range=(1, 3), stop_words="english", max_df=0.95, min_df=5)

# 2. Embedding model más potente
embedding_model = SentenceTransformer("all-mpnet-base-v2")  # más preciso pero más pesado

# 3. Reducción de dimensionalidad con PCA en lugar de UMAP
from sklearn.decomposition import PCA
dim_reduction_model = PCA(n_components=15)

# 4. Clustering con KMeans en lugar de HDBSCAN
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=20, random_state=42)

# 5. Crear el modelo BERTopic con estos componentes
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    umap_model=dim_reduction_model,
    hdbscan_model=clustering_model,
    language="english",
    calculate_probabilities=True,
    verbose=True
)

# 6. Entrenamiento del modelo
topics, probs = topic_model.fit_transform(docs)

# LDA

1. Preprocesar y vectorizar los documentos
LDA en sklearn requiere una matriz documento-palabra (podemos usar para esto CountVectorizer o TF-IDF).

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorización (podés ajustar stop_words, ngram_range, etc.)
vectorizer = CountVectorizer(stop_words='english')
doc_term_matrix = vectorizer.fit_transform(docs)

2. Después simplemente podemos ya entrenar el modelo

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

# Definir número de tópicos
n_topics = 10

lda_model = LatentDirichletAllocation(n_components=n_topics,
                                       max_iter=10,
                                       learning_method='online',
                                       random_state=42)
lda_model.fit(doc_term_matrix)

LatentDirichletAllocation(learning_method='online', random_state=42)

In [14]:
# Obtener las palabras del vocabulario
terms = vectorizer.get_feature_names_out()

# Mostrar los 10 términos más importantes por tópico
for idx, topic in enumerate(lda_model.components_):
    print(f"\nTópico {idx}:")
    top_indices = topic.argsort()[-10:][::-1]
    top_terms = [terms[i] for i in top_indices]
    print(", ".join(top_terms))


Tópico 0:
ax, max, b8f, g9v, a86, pl, 145, 1d9, 0t, 1t

Tópico 1:
00, 10, 25, 11, 12, 20, 15, 14, 16, 17

Tópico 2:
year, game, team, car, games, new, years, season, earth, hockey

Tópico 3:
file, windows, dos, edu, software, image, files, use, ftp, program

Tópico 4:
people, don, think, just, know, like, say, did, said, time

Tópico 5:
like, just, use, know, don, time, used, does, good, new

Tópico 6:
db, bh, radar, spot, mx, mk, mm, si, brake, ah

Tópico 7:
god, jesus, bible, church, christian, christ, christians, faith, lord, sin

Tópico 8:
government, armenian, israel, armenians, public, university, information, jews, national, new

Tópico 9:
edu, com, scsi, drive, tape, cs, controller, ide, bus, ra


In [15]:
# Distribución de tópicos por documento
doc_topic_dist = lda_model.transform(doc_term_matrix)

# Por ejemplo: tópico más probable para el primer documento
import numpy as np
print("Tópico dominante en el primer documento:", np.argmax(doc_topic_dist[0]))

Tópico dominante en el primer documento: 4
